In [1]:
ticker = "AAPL"
interval="1min"
APIKeys = ['PVTA8QOPS830VUM5','XO189HVC2WVKDQ1X','7L6O09DVXGIS3KFA','Y8SNZH5BAKUQ03B0','0RHBGT0RPA5Y5453','V5U9UHC52V5JCHSD','NTHZ6XUD2P9Q94WE','C8V7NVT8FF02BMX3'] #25 calls per day
#More could be obtained from here : https://www.alphavantage.co/support/#api-key

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# Define the start and end dates for the data
start_date = '2010-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')  # Current date as the end date

# Convert start_date and end_date to datetime objects
start = datetime.strptime(start_date, '%Y-%m-%d')
end = datetime.strptime(end_date, '%Y-%m-%d')

# Initialize the number of API calls and API key index
calls = 0
api_key_index = 0

# Alpha Vantage API function to fetch data
def fetch_alpha_vantage_data(symbol, interval, month_str):
    """
    Fetches intraday stock data from Alpha Vantage API for the given symbol, interval, and month.

    Args:
    symbol (str): Stock ticker symbol.
    interval (str): Data interval (e.g., '1min', '5min').
    month_str (str): Month in 'YYYY-MM' format.

    Returns:
    pd.DataFrame: DataFrame containing the stock data for the month.
    """
    global calls, api_key_index
    APIKey = APIKeys[api_key_index]  # Select the current API key
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&outputsize=full&apikey={APIKey}&month={month_str}"
    print(f"\nCalling {url}\n")

    response = requests.get(url)  # Make the API call

    # Increment the call counter
    calls += 1

    # Switch to the next API key after 25 calls (Alpha Vantage API limits)
    if calls == 25:
        api_key_index += 1
        calls = 0

    # Check if the API call was successful
    if response.status_code == 200:
        data = response.json()  # Parse the JSON response
        if 'Time Series (1min)' in data:
            # Create a DataFrame from the returned data
            df = pd.DataFrame.from_dict(data['Time Series (1min)'], orient='index')
            df.columns = ['open', 'high', 'low', 'close', 'volume']  # Set column names
            df.index = pd.to_datetime(df.index)  # Convert index to datetime
            return df
        else:
            print(f"No data returned for {month_str}. Retrying ..")
            return None
    else:
        # If there is an error, print the status code
        print(f"Error {response.status_code} from Alpha Vantage. Retrying ..")
        return None

# Initialize an empty DataFrame to store all fetched data
all_data = pd.DataFrame()

# Set the chunk size to approximately one month (31 days)
delta = timedelta(days=31)

# Start from the defined start date
current_start = start

# Loop through each month between the start and end dates
while current_start < end:
    current_end = current_start + delta  # Calculate the end date for the current chunk
    if current_end > end:
        current_end = end  # Adjust the end date if it exceeds the overall end date

    # Format the month as 'YYYY-MM' for the API request
    month_str = current_start.strftime('%Y-%m')

    print(f"Fetching data for {month_str}")

    # Fetch data for the current month
    data = fetch_alpha_vantage_data(ticker, interval, month_str)
    print(data)

    # Append the data to the master DataFrame if data was returned
    if data is not None and not data.empty:
        all_data = pd.concat([all_data, data])

    # Move to the next month
    current_start = current_end

    # Sleep for 12 seconds to respect the API rate limit (5 requests per minute for free-tier)
    time.sleep(5)

# Sort the data by the timestamp (since the API returns data in descending order)
all_data.sort_index(inplace=True)

# Save the final merged data to a CSV file
all_data.to_csv(f'{ticker}_minute_data_from_2010.csv')

# Confirm that the data has been saved
print(f"Data saved to {ticker}_minute_data_from_2010.csv")
